In [1]:
import pandas as pd
import folium
from haversine import haversine
import numpy as np
import seaborn as sns
import json
import os
from copy import deepcopy 
import geopandas as gpd
from shapely.geometry import Point
sns.set()

In [2]:
# Local
# Font
import matplotlib
from matplotlib import font_manager, rc
import platform
if platform.system()=="Windows":
    font_name=font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
matplotlib.rcParams['axes.unicode_minus']=False

import warnings
warnings.filterwarnings("ignore")

In [3]:
path = "D:\COMPAS\SBJ_2012_001"

In [9]:
total = gpd.read_file(os.path.join(path,'total.geojson'))
total.head(1)

,gid,accident_cnt,어린이집_count,유치원_count,초등학교_count,방지턱_count,불법주정차_cam_count,정류장_count_100,정류장_count_300,유동인구_sum,유소년인구_count_200,학원수_count_300,유소년인구_count_100,학원수_count_100,geometry
0,다사561083,1,0,0,0,0,0,2,2,43.435676,0.0,0,0.0,0,"MULTIPOLYGON (((127.00549 37.17243, 127.00549 ..."


# X ,Y 생성

In [10]:
X = pd.DataFrame(total.drop(["gid","accident_cnt","geometry"],axis=1))
cols = X.columns
Y = pd.Series(total["accident_cnt"])
X.shape , Y.shape

((345, 12), (345,))

# Train-Test split

In [11]:
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
m_scaler = MinMaxScaler()
X = scaler.fit_transform(X)

x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.25)
x_train.shape , x_test.shape

((258, 12), (87, 12))

# Linear Regressor

In [12]:
from sklearn.linear_model import LinearRegression

LR = LinearRegression()
LR.fit(x_train, y_train)

score = LR.score(x_test,y_test)
print(score)

0.013902144370005254


In [13]:
print(LR.predict(x_test[0].reshape(1,-1)))
print(LR.coef_)

[1.77685988]
[ 0.18314508  0.03755685  0.00848777  0.0356217  -0.07755756  0.07159313
 -0.01661443  0.1586965  -0.07861593  0.04516493 -0.11837979  0.10907059]


# AdaBoostRegressor

In [19]:
from sklearn.ensemble import AdaBoostRegressor

ada = AdaBoostRegressor(n_estimators=20,learning_rate=0.1,loss="square",random_state=42)
ada.fit(x_train, y_train)

score = ada.score(x_test,y_test)
print(score)

-0.022250404533341595


In [15]:
top3 = np.argsort(ada.feature_importances_)[::-1][:3]
print("Top3 주요 특성 :",cols[top3].values)

Top3 주요 특성 : ['유동인구_sum' '학원수_count_300' '어린이집_count']
